In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [2]:
URL = "https://www.travelandleisure.com/holiday-travel/halloween/most-haunted-places-in-the-world"

In [3]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)

In [4]:
webpage = requests.get(URL, headers=HEADER)
webpage

<Response [200]>

In [5]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup

In [6]:
places = soup.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
for i in range(20):
    print(str(i) + "  " + places[i].text)
#print(places[1].text)

0   Raynham Hall in Norfolk, England 
1   Bhangarh Fort in Rajasthan, India 
2   Poveglia in Venice, Italy 
3   The Ancient Ram Inn in Wotton-under-Edge, England 
4   Port Arthur in Tasmania, Australia 
5   The Queen Mary in Long Beach, California 
6   Burg Wolfsegg in Wolfsegg, Germany 
7   The Lizzie Borden House in Fall River, Massachusetts 
8   Casa Loma in Toronto, Canada 
9   Château de Brissac in Brissac-Quincé, France 
10   The Stanley Hotel in Estes Park, Colorado 
11   Fairmont Banff Springs in Alberta, Canada 
12   St. Augustine Lighthouse in St. Augustine, Florida 
13   One If by Land, Two If by Sea in New York City 
14   The Tower of London in the United Kingdom 
15   Dock Street Theatre in Charleston, South Carolina 
16   Tao Dan Park in Ho Chi Minh City, Vietnam 
17   The Old Vicarage in Borgvattnet, Sweden 
18   The Winchester Mystery House in San Jose, California 
19   The Driskill in Austin, Texas 


In [7]:
title_list = []
location_list = []
counter = 0

print(places[10].text)
place_list = places[10].text.strip().split(" ")
print(place_list)
for i in range(len(place_list)):
    if place_list[i] == "in":
        counter = 1
        continue
    elif place_list[i] != "in" and counter < 1:
        title_list.append(place_list[i])
    elif place_list[i] != "in" and counter == 1:
        location_list.append(place_list[i])
seperator = " "
title = seperator.join(title_list)
location = seperator.join(location_list)
print(location)
print(title)


location_description = geocoder.geocode(location)
#location_description

 The Stanley Hotel in Estes Park, Colorado 
['The', 'Stanley', 'Hotel', 'in', 'Estes', 'Park,', 'Colorado']
Estes Park, Colorado
The Stanley Hotel


In [8]:
try:
    location_description = geocoder.geocode(location)
except Exception:
    print("No data found for location: " + location)
    #continue
try:
    postcode = location_description[1]['components']['postcode'].replace(" ","-")
except:
    postcode = "Null"
try:
    city = location_description[0]['components']['city']
except:
    city = "Null"
country = location_description[0]['components']['country']
country_code = location_description[0]['components']['country_code']
continent = location_description[0]['components']['continent']
latitude = str(location_description[0]['geometry']['lat'])
longitude = str(location_description[0]['geometry']['lng'])
print(len(location_description))

[{'annotations': {'DMS': {'lat': "40° 20' 10.89240'' N",
    'lng': "105° 31' 51.86964'' W"},
   'FIPS': {'county': '08069', 'state': '08'},
   'MGRS': '13TDE5489065226',
   'Maidenhead': 'DN70fi60gr',
   'Mercator': {'x': -11747665.517, 'y': 4887279.161},
   'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=33121490#map=16/40.33636/-105.53107',
    'note_url': 'https://www.openstreetmap.org/note/new#map=16/40.33636/-105.53107&layers=N',
    'url': 'https://www.openstreetmap.org/?mlat=40.33636&mlon=-105.53107#map=16/40.33636/-105.53107'},
   'UN_M49': {'regions': {'AMERICAS': '019',
     'NORTHERN_AMERICA': '021',
     'US': '840',
     'WORLD': '001'},
    'statistical_groupings': ['MEDC']},
   'callingcode': 1,
   'currency': {'alternate_symbols': ['US$'],
    'decimal_mark': '.',
    'disambiguate_symbol': 'US$',
    'html_entity': '$',
    'iso_code': 'USD',
    'iso_numeric': '840',
    'name': 'United States Dollar',
    'smallest_denomination': 1,
    'subunit': 'Cent'

In [9]:
city_address = []
for i in range(20):
    
    # SEPERATING TITLE AND LOCATION
    title_list = []
    location_list = []
    counter = 0
    place_list = []
    
    place_list = places[i].text.strip().split(" ")
    #print(place_list)
    for x in range(len(place_list)):
        if place_list[x] == "in":
            counter = 1
            continue
        elif place_list[x] != "in" and counter < 1:
            title_list.append(place_list[x])
        elif place_list[x] != "in" and counter == 1:
            location_list.append(place_list[x])
    seperator = " "
    title = seperator.join(title_list)
    location = seperator.join(location_list)

    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(location)
    except Exception:
        print(str(i) + " " + "No data found for location: " + location)
        continue
    
    # GETTING ADDRESS VALUES
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
    
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ title + "  " + city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode + "\n")
    address_dict = {'Location': title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

[{'Location': 'Raynham Hall', 'City/State': 'England', 'Country': 'United Kingdom', 'Country Code': 'GB', 'Continent': 'Europe', 'Latitude': '52.67', 'Longitude': '1.0', 'Postcode': 'NR203PF'}, {'Location': 'Bhangarh Fort', 'City/State': 'Rajasthan', 'Country': 'India', 'Country Code': 'IN', 'Continent': 'Asia', 'Latitude': '26.81', 'Longitude': '73.77', 'Postcode': 'NULL'}, {'Location': 'Poveglia', 'City/State': 'Venice', 'Country': 'Italy', 'Country Code': 'IT', 'Continent': 'Europe', 'Latitude': '45.44', 'Longitude': '12.33', 'Postcode': '30125'}, {'Location': 'The Ancient Ram Inn', 'City/State': 'Wotton-under-Edge', 'Country': 'United Kingdom', 'Country Code': 'GB', 'Continent': 'Europe', 'Latitude': '51.64', 'Longitude': '-2.35', 'Postcode': 'GL127BU'}, {'Location': 'Port Arthur', 'City/State': 'Tasmania', 'Country': 'Australia', 'Country Code': 'AU', 'Continent': 'Oceania', 'Latitude': '-42.04', 'Longitude': '146.64', 'Postcode': 'NULL'}, {'Location': 'The Queen Mary', 'City/Stat

In [10]:
ca_most_haunted_df = pd.DataFrame.from_dict(city_address)
ca_most_haunted_df

,Location,City/State,Country,Country Code,Continent,Latitude,Longitude,Postcode
0,Raynham Hall,England,United Kingdom,GB,Europe,52.67,1.0,NR203PF
1,Bhangarh Fort,Rajasthan,India,IN,Asia,26.81,73.77,NULL
2,Poveglia,Venice,Italy,IT,Europe,45.44,12.33,30125
3,The Ancient Ram Inn,Wotton-under-Edge,United Kingdom,GB,Europe,51.64,-2.35,GL127BU
4,Port Arthur,Tasmania,Australia,AU,Oceania,-42.04,146.64,NULL
5,The Queen Mary,Long Beach,United States,US,North America,33.77,-118.19,90802
6,Burg Wolfsegg,Bavaria,Germany,DE,Europe,49.11,11.98,93195
7,The Lizzie Borden House,Fall River,United States,US,North America,41.7,-71.15,02722
8,Casa Loma,Old Toronto,Canada,CA,North America,43.65,-79.38,M5H2N2
9,Château de Brissac,Brissac-Loire-Aubance,France,FR,Europe,47.34,-0.44,49320
